## Jester Jokes Collaborative Filtering Rec. Sys
## Import Libraries

In [1]:
import numpy as np
import pandas as pd
from scipy import sparse
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity

## Import Preprocessed Jester Preprocessed Data

In [2]:
ratings = pd.read_csv("jester_data.csv", index_col=0)
ratings

,"A man visits the doctor. The doctor says ""I have bad news for you.You have cancer and Alzheimer's disease"". The man replies ""Well,thank God I don't have cancer!""","This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. ""What could they possibly have said to make you move out?"" ""They told me that you were a pedophile."" He replied, ""That's an awfully big word for a ten year old.""",Q. What's 200 feet long and has 4 teeth? A. The front row at a Willie Nelson Concert.,Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.,"Q.\tWhat's O. J. Simpson's Internet address? A.\tSlash, slash, backslash, slash, slash, escape.","Bill & Hillary are on a trip back to Arkansas. They're almost out of gas, so Bill pulls into a service station on the outskirts of town. The attendant runs out of the station to serve them when Hillary realizes it's an old boyfriend from high school. She and the attendant chat as he gases up their car and cleans the windows. Then they all say good-bye. As Bill pulls the car onto the road, he turns to Hillary and says, 'Now aren't you glad you married me and not him ? You could've been the wife of a grease monkey !' To which Hillary replied, 'No, Bill. If I would have married him, you'd be pumping gas and he would be the President !'",How many feminists does it take to screw in a light bulb? That's not funny.,Q. Did you hear about the dyslexic devil worshiper? A. He sold his soul to Santa.,"A country guy goes into a city bar that has a dress code, and the maitre d' demands he wear a tie. Discouraged, the guy goes to his car to sulk when inspiration strikes: He's got jumper cables in the trunk! So he wraps them around his neck, sort of like a string tie (a bulky string tie to be sure) and returns to the bar. The maitre d' is reluctant, but says to the guy, ""Okay, you're a pretty resourceful fellow, you can come in... but just don't start anything""!","Two cannibals are eating a clown, one turns to other and says: ""Does this taste funny to you?",...,"A Panda bear walks into a bar. Sits down at a table and orders a beer and a double cheeseburger. After he is finished eating, he pulls out a gun and rips the place with gunfire. Patrons scatter and dive under chairs and tables as the bear runs out the door. After ensuring that no one is hurt, the bartender races out the door, and calls after the bear ""What the hell did you do that for?"" The bear calls back, ""I'm a Panda bear. Look it up in the dictionary."" The bartender returns, pulls out his dictionary. panda : \Pan""da\, n. (Zo[""o]l.) A small Asiatic mammal (Ailurus fulgens) having fine soft fur. It is related to the bears, and inhabits the mountains of Northern India. Eats shoots and leaves.","Early one morning a mother went to her sleeping son and woke him up. ""Wake up, son. It's time to go to school."" ""But why, Mama? I don't want to go to school."" ""Give me two reasons why you don't want to go to school."" ""One, all the children hate me. Two, all the teachers hate me,"" ""Oh! that's no reason. Come on, you have to go to school,"" ""Give me two good reasons WHY I should go to school?"" ""One, you are fifty-two years old. Two, you are the principal of the school.""","Reaching the end of a job interview, the human resources person asked a young engineer fresh out of Stanford, ""And what starting salary were you looking for?"" The engineer said, ""In the neighborhood of $125,000 a year, depending on the benefits package."" The interviewer said, ""Well, what would you say to a package of 5-weeks vacation, 14 paid holidays, full medical and dental, company matching retirement fund to 50% of salary, and a company car leased every 2 years - say, a red Corvette?"" The Engineer sat up straight and said, ""Wow! Are you kidding?"" And the interview

In [3]:
# Change data values to numeric values
ratings= ratings.apply(pd.to_numeric)
ratings

,"A man visits the doctor. The doctor says ""I have bad news for you.You have cancer and Alzheimer's disease"". The man replies ""Well,thank God I don't have cancer!""","This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. ""What could they possibly have said to make you move out?"" ""They told me that you were a pedophile."" He replied, ""That's an awfully big word for a ten year old.""",Q. What's 200 feet long and has 4 teeth? A. The front row at a Willie Nelson Concert.,Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.,"Q.\tWhat's O. J. Simpson's Internet address? A.\tSlash, slash, backslash, slash, slash, escape.","Bill & Hillary are on a trip back to Arkansas. They're almost out of gas, so Bill pulls into a service station on the outskirts of town. The attendant runs out of the station to serve them when Hillary realizes it's an old boyfriend from high school. She and the attendant chat as he gases up their car and cleans the windows. Then they all say good-bye. As Bill pulls the car onto the road, he turns to Hillary and says, 'Now aren't you glad you married me and not him ? You could've been the wife of a grease monkey !' To which Hillary replied, 'No, Bill. If I would have married him, you'd be pumping gas and he would be the President !'",How many feminists does it take to screw in a light bulb? That's not funny.,Q. Did you hear about the dyslexic devil worshiper? A. He sold his soul to Santa.,"A country guy goes into a city bar that has a dress code, and the maitre d' demands he wear a tie. Discouraged, the guy goes to his car to sulk when inspiration strikes: He's got jumper cables in the trunk! So he wraps them around his neck, sort of like a string tie (a bulky string tie to be sure) and returns to the bar. The maitre d' is reluctant, but says to the guy, ""Okay, you're a pretty resourceful fellow, you can come in... but just don't start anything""!","Two cannibals are eating a clown, one turns to other and says: ""Does this taste funny to you?",...,"A Panda bear walks into a bar. Sits down at a table and orders a beer and a double cheeseburger. After he is finished eating, he pulls out a gun and rips the place with gunfire. Patrons scatter and dive under chairs and tables as the bear runs out the door. After ensuring that no one is hurt, the bartender races out the door, and calls after the bear ""What the hell did you do that for?"" The bear calls back, ""I'm a Panda bear. Look it up in the dictionary."" The bartender returns, pulls out his dictionary. panda : \Pan""da\, n. (Zo[""o]l.) A small Asiatic mammal (Ailurus fulgens) having fine soft fur. It is related to the bears, and inhabits the mountains of Northern India. Eats shoots and leaves.","Early one morning a mother went to her sleeping son and woke him up. ""Wake up, son. It's time to go to school."" ""But why, Mama? I don't want to go to school."" ""Give me two reasons why you don't want to go to school."" ""One, all the children hate me. Two, all the teachers hate me,"" ""Oh! that's no reason. Come on, you have to go to school,"" ""Give me two good reasons WHY I should go to school?"" ""One, you are fifty-two years old. Two, you are the principal of the school.""","Reaching the end of a job interview, the human resources person asked a young engineer fresh out of Stanford, ""And what starting salary were you looking for?"" The engineer said, ""In the neighborhood of $125,000 a year, depending on the benefits package."" The interviewer said, ""Well, what would you say to a package of 5-weeks vacation, 14 paid holidays, full medical and dental, company matching retirement fund to 50% of salary, and a company car leased every 2 years - say, a red Corvette?"" The Engineer sat up straight and said, ""Wow! Are you kidding?"" And the interview

## Generate a utility matrix to extract similarities among jokes
###### For this I have used Pandas built in Corr() method that will compute a correlation score for every column pair in the matrix! This gives us a correlation score between every pair of jokes (I take into consideration that at least one user rated both jokes - otherwise NaN's would show up). Here I could also standardize the NaN scores and use the cosine_similarity method to calculate similarities between the jokes. However I prefered to use the pandas method because of it's simplicity.

In [4]:
#first corr() buildup
similarity_matrix = ratings.corr()
similarity_matrix

,"A man visits the doctor. The doctor says ""I have bad news for you.You have cancer and Alzheimer's disease"". The man replies ""Well,thank God I don't have cancer!""","This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. ""What could they possibly have said to make you move out?"" ""They told me that you were a pedophile."" He replied, ""That's an awfully big word for a ten year old.""",Q. What's 200 feet long and has 4 teeth? A. The front row at a Willie Nelson Concert.,Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.,"Q.\tWhat's O. J. Simpson's Internet address? A.\tSlash, slash, backslash, slash, slash, escape.","Bill & Hillary are on a trip back to Arkansas. They're almost out of gas, so Bill pulls into a service station on the outskirts of town. The attendant runs out of the station to serve them when Hillary realizes it's an old boyfriend from high school. She and the attendant chat as he gases up their car and cleans the windows. Then they all say good-bye. As Bill pulls the car onto the road, he turns to Hillary and says, 'Now aren't you glad you married me and not him ? You could've been the wife of a grease monkey !' To which Hillary replied, 'No, Bill. If I would have married him, you'd be pumping gas and he would be the President !'",How many feminists does it take to screw in a light bulb? That's not funny.,Q. Did you hear about the dyslexic devil worshiper? A. He sold his soul to Santa.,"A country guy goes into a city bar that has a dress code, and the maitre d' demands he wear a tie. Discouraged, the guy goes to his car to sulk when inspiration strikes: He's got jumper cables in the trunk! So he wraps them around his neck, sort of like a string tie (a bulky string tie to be sure) and returns to the bar. The maitre d' is reluctant, but says to the guy, ""Okay, you're a pretty resourceful fellow, you can come in... but just don't start anything""!","Two cannibals are eating a clown, one turns to other and says: ""Does this taste funny to you?",...,"A Panda bear walks into a bar. Sits down at a table and orders a beer and a double cheeseburger. After he is finished eating, he pulls out a gun and rips the place with gunfire. Patrons scatter and dive under chairs and tables as the bear runs out the door. After ensuring that no one is hurt, the bartender races out the door, and calls after the bear ""What the hell did you do that for?"" The bear calls back, ""I'm a Panda bear. Look it up in the dictionary."" The bartender returns, pulls out his dictionary. panda : \Pan""da\, n. (Zo[""o]l.) A small Asiatic mammal (Ailurus fulgens) having fine soft fur. It is related to the bears, and inhabits the mountains of Northern India. Eats shoots and leaves.","Early one morning a mother went to her sleeping son and woke him up. ""Wake up, son. It's time to go to school."" ""But why, Mama? I don't want to go to school."" ""Give me two reasons why you don't want to go to school."" ""One, all the children hate me. Two, all the teachers hate me,"" ""Oh! that's no reason. Come on, you have to go to school,"" ""Give me two good reasons WHY I should go to school?"" ""One, you are fifty-two years old. Two, you are the principal of the school.""","Reaching the end of a job interview, the human resources person asked a young engineer fresh out of Stanford, ""And what starting salary were you looking for?"" The engineer said, ""In the neighborhood of $125,000 a year, depending on the benefits package."" The interviewer said, ""Well, what would you say to a package of 5-weeks vacation, 14 paid holidays, full medical and dental, company matching retirement fund to 50% of salary, and a company car leased every 2 years - say, a red Corvette?"" The Engineer sat up straight and said, ""Wow! Are you kidding?"" And the interview

###### To avoid spurious results that happened from just a handful of users that happened to rate the same pair of jokes. In order to restrict our results to jokes that lots of people rated together - and also give more popular results that are more easily recongnizable - the min_periods argument was used to throw out results where fewer than 100 users rated a given jokes pair. The rating average per user could be used to determine the min_periods argument however given the dataset I decisded to use the entire content:

In [5]:
similarity_matrix = ratings.corr(method='pearson', min_periods=100)
similarity_matrix

,"A man visits the doctor. The doctor says ""I have bad news for you.You have cancer and Alzheimer's disease"". The man replies ""Well,thank God I don't have cancer!""","This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him. ""What could they possibly have said to make you move out?"" ""They told me that you were a pedophile."" He replied, ""That's an awfully big word for a ten year old.""",Q. What's 200 feet long and has 4 teeth? A. The front row at a Willie Nelson Concert.,Q. What's the difference between a man and a toilet? A. A toilet doesn't follow you around after you use it.,"Q.\tWhat's O. J. Simpson's Internet address? A.\tSlash, slash, backslash, slash, slash, escape.","Bill & Hillary are on a trip back to Arkansas. They're almost out of gas, so Bill pulls into a service station on the outskirts of town. The attendant runs out of the station to serve them when Hillary realizes it's an old boyfriend from high school. She and the attendant chat as he gases up their car and cleans the windows. Then they all say good-bye. As Bill pulls the car onto the road, he turns to Hillary and says, 'Now aren't you glad you married me and not him ? You could've been the wife of a grease monkey !' To which Hillary replied, 'No, Bill. If I would have married him, you'd be pumping gas and he would be the President !'",How many feminists does it take to screw in a light bulb? That's not funny.,Q. Did you hear about the dyslexic devil worshiper? A. He sold his soul to Santa.,"A country guy goes into a city bar that has a dress code, and the maitre d' demands he wear a tie. Discouraged, the guy goes to his car to sulk when inspiration strikes: He's got jumper cables in the trunk! So he wraps them around his neck, sort of like a string tie (a bulky string tie to be sure) and returns to the bar. The maitre d' is reluctant, but says to the guy, ""Okay, you're a pretty resourceful fellow, you can come in... but just don't start anything""!","Two cannibals are eating a clown, one turns to other and says: ""Does this taste funny to you?",...,"A Panda bear walks into a bar. Sits down at a table and orders a beer and a double cheeseburger. After he is finished eating, he pulls out a gun and rips the place with gunfire. Patrons scatter and dive under chairs and tables as the bear runs out the door. After ensuring that no one is hurt, the bartender races out the door, and calls after the bear ""What the hell did you do that for?"" The bear calls back, ""I'm a Panda bear. Look it up in the dictionary."" The bartender returns, pulls out his dictionary. panda : \Pan""da\, n. (Zo[""o]l.) A small Asiatic mammal (Ailurus fulgens) having fine soft fur. It is related to the bears, and inhabits the mountains of Northern India. Eats shoots and leaves.","Early one morning a mother went to her sleeping son and woke him up. ""Wake up, son. It's time to go to school."" ""But why, Mama? I don't want to go to school."" ""Give me two reasons why you don't want to go to school."" ""One, all the children hate me. Two, all the teachers hate me,"" ""Oh! that's no reason. Come on, you have to go to school,"" ""Give me two good reasons WHY I should go to school?"" ""One, you are fifty-two years old. Two, you are the principal of the school.""","Reaching the end of a job interview, the human resources person asked a young engineer fresh out of Stanford, ""And what starting salary were you looking for?"" The engineer said, ""In the neighborhood of $125,000 a year, depending on the benefits package."" The interviewer said, ""Well, what would you say to a package of 5-weeks vacation, 14 paid holidays, full medical and dental, company matching retirement fund to 50% of salary, and a company car leased every 2 years - say, a red Corvette?"" The Engineer sat up straight and said, ""Wow! Are you kidding?"" And the interview

###### I manually added to the data set a user to serve as test case to produce some recommendation system using the user ID. The results of his ratings to the jokes are bellow:

In [6]:
myRatings = ratings.loc['User4'].dropna()
myRatings

   This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him.  "What could they possibly have said to make you move out?"  "They told me that you were a pedophile."  He replied, "That's an awfully big word for a ten year old."                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

###### Now let's look at each joke the user rated one at a time, and build up a possible recommendation based on the jokes similar to the ones he rated. So for each rated joke by the user, it will be retrieved the list of similar movies from the correlation matrix. It will then scale those correlation scores by how well rated the joke was. so jokes similar to the one that was rated by the user count more than jokes that he hated: 

In [7]:
simCandidates = pd.Series()
for i in range(0, len(myRatings.index)):
    print("Adding sims for " + myRatings.index[i] + "...")
    # Retriving similar jokes to the one that was rated
    sims = similarity_matrix[myRatings.index[i]].dropna()
    # Scaling its similarity by how well the joke was rated
    sims = sims.map(lambda x: x * myRatings[i])
    # Adding the score to the list of similarity candidates
    simCandidates = simCandidates.append(sims)
    
# Results
print("Sorting...")
simCandidates.sort_values(inplace = True, ascending = False)
print(simCandidates.head(10))

Adding sims for    This couple had an excellent relationship going until one day he came home from work to find his girlfriend packing. He asked her why she was leaving him and she told him that she had heard awful things about him.  "What could they possibly have said to make you move out?"  "They told me that you were a pedophile."  He replied, "That's an awfully big word for a ten year old." ...
Adding sims for    Q.	What's O. J. Simpson's Internet address?  A.	Slash, slash, backslash, slash, slash, escape....
Adding sims for    Bill & Hillary are on a trip back to Arkansas. They're almost out of gas, so Bill pulls into a service station on the outskirts of town. The attendant runs out of the station to serve them when Hillary realizes it's an old boyfriend from high school. She and the attendant chat as he gases up their car and cleans the windows. Then they all say good-bye.  As Bill pulls the car onto the road, he turns to Hillary and says, 'Now aren't you glad you married me and

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


###### After getting more useful information, note that some of the same jokes come up more than once, because they were similar to more than one joke that was rated. I used groupby() to add together the scores from jokes that show up more then once, so they will have a higher weight:

In [8]:
simCandidates = simCandidates.groupby(simCandidates.index).sum()

In [9]:
simCandidates.sort_values(inplace = True, ascending = False)
simCandidates.head(10)

   The father was very anxious to marry off his only daughter so he wanted  to impress her date. "Do  you like to screw," he says. "Huh" replied the  surprised first date. "My daughter she loves to screw and she's good at it,  you and her  should go screw," carefully explained the father. Now very  interested the boy replied, "Yes, sir." Minutes later the girl came down  the stairs, kissed her father goodbye and the couple left. After only a  few minutes she reappeared, furious, dress torn, hair a mess and screamed "Dammit, Daddy, it's the TWIST, get it straight!"                                                                                                                                                                    59.816260
   A guy walks into a bar and sits down next to an extremely gorgeous  woman.  The first thing he notices about her though, are her pants.   They were skin-tight, high-waisted and had no obvious mechanism  (zipper, buttons or velcro) for opening them. Afte

## Recommendation
###### The last thing that needs to be done is filter out jokes that where rated by the user, because it's not useful to recommend a joke someone already read and ranked.

In [10]:
print('==========******=============')
print('TOP RECOMMENDED JOKES FOR YOU')
print('==========******=============')
filteredSims = simCandidates.drop(myRatings.index)
filteredSims.head(10)

==========******=============
TOP RECOMMENDED JOKES FOR YOU
==========******=============


   Did you hear that Clinton has announced there is a new national bird?   The spread eagle.                                                                                                                                                                                                                                                                                                                                                                                                                                                        52.047342
   Two kindergarten girls were talking outside: one said, "You won't believe what I saw on the patio yesterday--a condom!" The second girl asked, "What's a patio?"                                                                                                                                                                                                                                                                                                       